In [110]:
import sqlite3
import pandas as pd
import numpy as np
! ls

API Mini Project.ipynb          SQLTasks_Tier2_Q10-13.ipynb
GuidedCapstone                  SQLTasks_Tier_2.sql
Notes_from_DataCamp             Unit 4 Challenge - Tier 3.ipynb
README.md                       data.sqlite
SB_DB.sql                       meteorites.ipynb


In [111]:
#phpMyAdmin is SQL, the exported file needs to be converted to sqlite.
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

### Q10: Produce a list of facilities with a total revenue less than 1000. 
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 

In [112]:
# Annoying: memid = 'str', not int
guest_bookings = pd.read_sql_query("""
SELECT facid, SUM(slots) AS g_tot 
FROM Bookings 
WHERE memid = '0' 
GROUP BY facid;""", conn, index_col = 'facid')

In [216]:
member_bookings = pd.read_sql_query("""
SELECT facid, SUM(slots) AS Member_Bookings 
FROM Bookings 
WHERE memid <> '0' 
GROUP BY facid;""", conn, index_col = 'facid')

In [114]:
cost = pd.read_sql_query("""
SELECT facid, guestcost, membercost, name
FROM  Facilities  
GROUP BY facid;
""", conn, index_col = 'facid')

In [124]:
# Cost is an object. Change to numeric
#cost.info()
cost['guestcost'] = cost['guestcost'].astype('float')
cost['membercost'] = cost['membercost'].astype('float')

In [117]:
# Get revenue by guest bookings * guest costs and add to member bookings * member costs
cost['Revenue'] = np.multiply(guest_bookings['g_tot'],cost['guestcost']) + np.multiply(member_bookings['Member_Bookings'],cost['membercost']).astype(float)

In [123]:
# Format the table. Select only facilities with revenue < $1000 and sort by revenue
Cost = cost[cost['Revenue'] < 1000].drop(columns = ['guestcost','membercost']).sort_values(by = 'Revenue', ascending=False)
Cost

,name,Revenue
facid,,
8,Pool Table,270.0
7,Snooker Table,240.0
3,Table Tennis,180.0


### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 


In [205]:
member_rec = pd.read_sql_query("""SELECT surname, firstname, recommendedby 
FROM  Members
WHERE recommendedby > '0'
ORDER BY surname, firstname""",conn)

In [206]:
member_rec.head()

,surname,firstname,recommendedby
0,Bader,Florence,9
1,Baker,Anne,9
2,Baker,Timothy,13
3,Boothe,Tim,3
4,Butters,Gerald,1


If we want the recommendedby with a name:

In [207]:
#Query to get the names and member id relationship
member_id = pd.read_sql_query(""" 
SELECT memid, firstname || ' ' || surname AS member_name 
FROM Members 
""", conn)

In [208]:
# Create a dictionary 
member_lookup = dict(zip(member_id['memid'],member_id['member_name']))

# Transform the column
member_rec['recommendedby'] = member_rec['recommendedby'].map(member_lookup)

In [209]:
member_rec.head()

,surname,firstname,recommendedby
0,Bader,Florence,Ponder Stibbons
1,Baker,Anne,Ponder Stibbons
2,Baker,Timothy,Jemima Farrell
3,Boothe,Tim,Tim Rownam
4,Butters,Gerald,Darren Smith


### Q12: Find the facilities with their usage by member, but not guests

In [217]:
member_bookings.insert(0, 'Facility', cost['name'])
member_bookings

,Facility,Member_Bookings
facid,,
0,Tennis Court 1,957
1,Tennis Court 2,882
2,Badminton Court,1086
3,Table Tennis,794
4,Massage Room 1,884
5,Massage Room 2,54
6,Squash Court,418
7,Snooker Table,860
8,Pool Table,856


### Q13: Find the facilities usage by month, but not guests